# Multiple linear regression

## Import the relevant libraries

In [1]:
import numpy as np
from datetime import datetime
import time
from scipy.stats import norm
import numpy.linalg as linalg

## Load the data

In [2]:
#Generación e matriz aleatoria arbitraria
nobs=20
nvar=5
calibration_matrix=np.random.random((nobs,nvar)) #matriz de 300x112
#comprobar
print "Las dimensiones de la matriz de calibración son: ", calibration_matrix.shape


Las dimensiones de la matriz de calibración son:  (20, 5)


## Información adicional para el cálculo

In [3]:
#Configuraciones
tstsDateFormat='%Y%m%d%H%M'
model_backup_path = "data/calibration/"
datetime.strftime(datetime.now(),tstsDateFormat)

#información del yaml de configuración principal
prep=2
lv=2
phase=1
x=calibration_matrix


### Preprocesamiento de la matriz calibración

In [4]:
#preprocesamiento -->method_name = "preprocess_2D()" La variables weight de momento en estos ejemplos no la tendremos en cuenta
nanM = np.isnan(x)
anM = 1 - nanM

average = np.nanmean(x,axis=0)# array of M elements
average = average.reshape((1,average.shape[0]))# Matrix of 1xM elements
scale = np.nanstd(x,axis=0,ddof=1)

#TODO: to ask Pepe what is this :(
ind = np.nonzero(scale == 0)# # of zeroes in scale
dem = 2.0*np.sum(anM[:,ind],axis=0) - 1
scale[ind] = np.sqrt(np.ones((1,np.array(ind).size)) / dem)

scale = scale.reshape((1,scale.shape[0]))# Matrix of 1xM elements
xcs = x - np.dot(np.ones((x.shape[0],1)),average)
xcs = xcs / np.dot(np.ones((x.shape[0],1)),scale)

print "average: ", average
print ("Las dimensiones de average: ", average.shape, " Se busca el valor medio de csda")#112x1, es decir, una media por cada columna

print "scale: ", scale
print "Las dimensiones de scale son: ", scale.shape, "Se busca la desviación de cada columna para centrarla en 1"

#xcs tiene las dimensiones de la matriz original 300x112
print "xcs: ",xcs
print "Las dimensiones de xcs: ", xcs.shape
print "xcs es la matriz normalizada, con media 0 y desviación 1, paso necesario para aplicar PCA"


average:  [[0.47191628 0.47270911 0.4020386  0.44044764 0.59037733]]
('Las dimensiones de average: ', (1, 5), ' Se busca el valor medio de csda')
scale:  [[0.3226111  0.30336455 0.26917673 0.26260354 0.31146312]]
Las dimensiones de scale son:  (1, 5) Se busca la desviación de cada columna para centrarla en 1
xcs:  [[-0.4893417   0.08297315 -0.66809013 -0.33776272  1.10974022]
 [ 0.36624061  1.54673393 -0.65550962 -0.28893527 -0.01272692]
 [ 1.19262797 -1.50587262 -0.90470108 -0.83499918 -1.31171509]
 [-1.26093473 -0.80659027  0.36997291 -0.44652472  0.41583683]
 [ 0.87946556  0.4733678  -0.87363655  0.16788317  0.58604103]
 [-0.53145919 -1.26925775 -0.52679628 -1.36211182  1.17541713]
 [-1.24592866  0.30930469 -0.03298969 -0.37503962 -0.45278567]
 [-0.98770838 -0.23458856 -0.92724663 -0.23604534  0.88093354]
 [-0.26304801 -0.93999737 -0.63828384  1.60057898  0.99684988]
 [ 1.40950777  0.60473813 -0.97094784  0.62929349 -0.54693542]
 [ 0.92105691  0.22054149 -1.3993101   1.59882626  1.2

### PCA

In [5]:
#PCA; La llamada de la funcion def runPCA(self, method='svd', **kwargs):
#El ejemplo lo describo con el método svd que es el usado por defecto
# Run SVD from the data matrix
data=xcs
pcs=lv

u, s, v = linalg.svd(data)

# Transform S from array to matrix with the corresponding dimensions
# in U and V
#Este punto no he conseguido obtener una comprensión profunda mas alla de lo que menciona
sdiag = np.diag(s)
s = np.zeros((u.shape[1], v.shape[0]))
s[:sdiag.shape[0], :sdiag.shape[1]] = sdiag

print "Las dimensiones de u: %s" % str(u.shape)

print "Las dimensiones de s: %s" % str(s.shape)

print "Las dimensiones de v: %s" % str(v.shape)


Las dimensiones de u: (20, 20)
Las dimensiones de s: (20, 5)
Las dimensiones de v: (5, 5)


In [6]:
t = np.dot(u, s)   #  ->u: Matriz de similaridad de fila-clase       
p = v.T #            ->v: Matriz de similaridad de columna-clase

scoresMatrix = t[:, :pcs]  # (Las dos pc's)
loadingsMatrix = p[:, :pcs] #M
model = np.dot(scoresMatrix, loadingsMatrix.T) #Matriz de 300x112
residualsMatrix = data - model #
eigengvaluesMatrix = s #

print "La dimensiones de t son : %s" % str(t.shape)
print "Las dimensiones de p son: %s" % str(p.shape)
print "Las dimensiones de scoresMatrix son: %s" % str(scoresMatrix.shape)
print "Las dimensiones de loadingsMatrix son: %s" % str(loadingsMatrix.shape)
print "Las dimensiones de model son: %s" % str(model.shape)
print "Las dimensiones de residualsMatrix son: %s" % str(residualsMatrix.shape)
print "Las dimensiones de eigengvaluesMatrix son: %s" % str(eigengvaluesMatrix.shape)


La dimensiones de t son : (20, 5)
Las dimensiones de p son: (5, 5)
Las dimensiones de scoresMatrix son: (20, 2)
Las dimensiones de loadingsMatrix son: (5, 2)
Las dimensiones de model son: (20, 5)
Las dimensiones de residualsMatrix son: (20, 5)
Las dimensiones de eigengvaluesMatrix son: (20, 5)


## Calculamos los UCL's

In [7]:
#definido en model.py
alpha = 0.01


####Para el calculo de UCLD
phase=2
mspc=lv
npc=data.shape[0] #nº observaciones (300)

### UCL-D

In [18]:
####Calculo de los UCL's
#UCL-D
#Definición
#def computeUCLD(self,npc,nob,p_value,phase):
#llamada
#self._mspc.computeUCLD(self._lv, data.shape[0], self._alpha, self._phase)
'''
Parameters
        ----------
        npc: int 
            Number of PCs
        nob: int 
            Number of observations
        p_value: float 
            p-value of the test, in (0,1]
        phase: int 
            SPC phase
            1: Phase I
            2: Phase II
'''
#method_name = "computeUCLD()"
       
#importar
from scipy.stats import f as fisher

####Para el calculo de UCLD
phase=2
npc=lv
nob=data.shape[0]#nº observaciones (300)
p_value=alpha #El p_value vale 0.01 muy significativo. A más pequeño más significativo. Basicamente busca le decimos a q que busque anomalías en el p-value dado

        
if phase == 2:
   lim = (npc*(nob*nob-1.0)/(nob*(nob-npc)))*fisher.ppf(1-p_value,npc,nob-npc)
else:
   lim = (nob-1.0)**2/nob*beta.ppf(1-p_value,npc/2.0,(nob-npc-1)/2.0)
         
print "El valor de lim es: %s" %lim
   
# Check is the limit is and ndarray of [1x1] dimensions and get the float value
if isinstance(lim, np.ndarray):
   lim = lim[0,0]
            
# TODO: Sometimes after computations numpy takes UCLq as complex with 0j imaginary part
if isinstance(lim, complex):
   logging.warn("UCLd has a complex value of %s. Getting just the real part.",lim)
   lim = lim.real
            
UCLD=lim #Valor de ejemplo 9.416825881055232

print UCLD


El valor de lim es: 13.328605717141173
13.328605717141173


### UCL-Q

In [9]:
#definido en model.py
alpha = 0.01

####Para el calculo de UCLQ
res=residualsMatrix
p_value=alpha #El p_value vale 0.01 muy significativo. A más pequeño más significativo. Basicamente busca le decimos a q que busque anomalías en el p-value dado
# Rows of E matrix
N = res.shape[0] #300 observaciones
            
# rank of E
pcs_left = np.linalg.matrix_rank(res);#rango 110, este valor variará según la matriz utilizada
    
#
lambda_eig = np.linalg.eigvals((1.0/(N-1))*np.dot(res.T,res)) #Array de 112 valores, 
# Get the DESC order according to the ABS value of eigenvalues
lambda_eig = lambda_eig[np.abs(lambda_eig).argsort()[::-1]]        
    
theta1 = np.sum(lambda_eig[:pcs_left])
theta2 = np.sum(lambda_eig[:pcs_left]**2)
theta3 = np.sum(lambda_eig[:pcs_left]**3)
    
h0 = 1-((2*theta1*theta3)/(3*theta2**2))
    
z = norm.ppf(1-p_value)

UCLq = theta1*((z*np.sqrt(2*theta2*(h0**2))/theta1) + 1 + (theta2*h0*(h0-1)/(theta1**2)))**(1/h0)
#Comprobaciones para evitar errorewres
# Check is the limit is and ndarray of [1x1] dimensions and get the float value
if isinstance(UCLq, np.ndarray):
   UCLq = UCLq[0,0]
                
# TODO: Sometimes after computations numpy takes UCLq as complex with 0j imaginary part
if isinstance(UCLq, complex):
    logging.warn("UCLq has a complex value of %s. Getting just the real part.",UCLq)
    UCLq = UCLq.real

UCLQ=UCLq #valor de ejemplo 150.99231878043952

print UCLQ

9.079596700175381


## Cálculo de los estadísticos

### Q-st

In [10]:
#Comenzamos con el cálculo de Q-st
#Primero obtenemos un arry de 1x112
test=np.random.random((1,5))
#La preprocesamos, es una funcion distinta a la usada en la matriz de calibración
#Es decir media 0 y desviación 1 respecto la calibración
testMeanCenterting = test - np.dot(np.ones((test.shape[0],1)),average)
testAutoScaled = testMeanCenterting / (np.dot(np.ones((test.shape[0],1)),scale))
testcs = testAutoScaled
#Ahora comenzamos el cálculo del estadístico, ¿Qué necesitamos?
#self._mspc.computeQst(testcs, self._model.get_pca().getLoadings())
#La observación preporcesada y la matriz Loading del PCA

#def computeQst(self,testcs,P):
P=loadingsMatrix
#new scores from testcs and the loadings (Q) of the calibration model
t = np.dot(testcs,P)
# Model residuals from the observations in testcs
e = testcs - np.dot(t,np.transpose(P))
# Computes Q-statistics from the observations in testcs
Qst = np.sum(np.power(e,2),axis=1).reshape(testcs.shape[0],1);
#->Qst = 89.84502164

print Qst

[[3.80170218]]


### D-st

In [11]:
#Partiendo de las ejecuciones anteriores cn ya test definido e incluso
"""
Parameters
        ----------        
        testcs: numpy.ndarray
            [NxM] preprocessed billinear data set with the observations to be monitored.
        P: numpy.ndarray 
            [MxA] Matrix to perform the projection from the original to the latent subspace. 
            For PCA (testcs = T*P'), this is the matrix of loadings
        T: numpy.ndarray 
            [MxA] Matrix to perform the projection from the original to the latent subspace. 
            For PCA (testcs = T*P'), this is the matrix of scores
"""
testcs=testcs
P=loadingsMatrix
T=scoresMatrix
#new scores from testcs and the loadings (R) of the calibration model
t = np.dot(testcs,P)        
            
#inverse of the model calibration scores (T)
#Note: inv() method just allows at least 2D arrays 
t_cov = np.cov(T,rowvar=False)

try:
	invCT = np.linalg.inv(t_cov)
except LinAlgError:
	invCT = 1 / t_cov

#         if all(t_cov.shape):# True the shape tuple is empty
#             # When T has only one variable -> cov(T) computes the variance
#             invCT = 1 / t_cov
#         else:    
#             invCT = np.linalg.inv(t_cov)

dotAux = np.dot(t,invCT)

# Computes D-statistics from the observations in testcs
Dst = np.sum(np.multiply(dotAux,t),axis=1).reshape(testcs.shape[0],1);
            
# Check is the statistic is and ndarray of [1x1] dimensions and get the float value
if isinstance(Dst, np.ndarray):
	Dst = Dst[0,0]
            
# TODO: Sometimes after computations numpy takes UCLq as complex with 0j imaginary part
if isinstance(Dst, complex):
	logging.warn("Dst has a complex value of %s. Getting just the real part.",self._Dst)
	Dst = Dst.real
    
print Dst

2.219034581647613


En este punto tenemos calculado los UCL's correspondientes a una calibración y los estadísticos correspondientes a una observación. Para comprobar si tenermos una anomalía tan sólo debemos ver si ambos estadísticos se encuentran por debajo de sus UCL's respectivos.

## Diagnóstico

El punto ahora es poder obtener el origen de dicha anomalía para ello se aplica el método oMEDA calculando un vector de diagnóstico de la siguiente manera:

In [12]:
# Set up which observations are compared
dummy = np.zeros((1,test.shape[0]))
# We evaluate the observation 1
dummy[0,0] = 1

# Computes oMEDA
#self._mspc.computeoMEDA(testcs, dummy, self._model.get_pca().getLoadings())
if dummy.shape[0] == 1:
  dummy = dummy.T
                            
# To normalice the dummy vector [-1, 1]
if dummy[dummy > 0].size != 0:
  dummy[dummy > 0] = dummy[dummy > 0] / np.max(dummy[dummy > 0])


if dummy[dummy < 0].size != 0:
  dummy[dummy < 0] = (dummy[dummy < 0] / np.min(dummy[dummy < 0]))*(-1)
                  
xA = np.dot(np.dot(testcs,P),P.T);   
sumA = np.dot(xA.T,dummy);       
sumTotal = np.dot(testcs.T,dummy);        
            
oMEDA = ((2*sumTotal - sumA)*np.abs(sumA)) / np.sqrt(np.dot(dummy.T,dummy))

diagnosis_vec=oMEDA

diagnosis_vec

array([[-1.73423996],
       [ 0.5620792 ],
       [ 1.32848689],
       [-0.66446484],
       [-0.08181854]])